<a href="https://colab.research.google.com/github/kunal-kumar-chaudhary/fine_tuning_codellama/blob/main/Fine_Tuning_codellama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 10.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

In [ ]:
!pip install accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, GenerationConfig, BitsAndBytesConfig, AutoConfig
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [ ]:
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

In [ ]:
dataset_name = "CM/codexglue_code2text_javascript"
dataset = load_dataset(dataset_name)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/58025 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3885 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3291 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 58025
    })
    validation: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 3885
    })
    test: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 3291
    })
})

In [ ]:
model_name = "codellama/CodeLlama-7b-hf"

# initializing quantization config
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True
)

# loading the configuration
config = AutoConfig.from_pretrained(model_name)

# initializing an empty model
with init_empty_weights():
  model = AutoModelForCausalLM.from_config(
        config,
        torch_dtype=torch.bfloat16, # for better precision
    )

# loading the actual model weights
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=config,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)


# loading the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
# let's see the number of trainable parameters
def count_parameters(model):
  trainable_model_params = 0
  all_model_params = 0
  for _, param in model.named_parameters():
    all_model_params += param.numel()
    if param.requires_grad:
      trainable_model_params += param.numel()
  return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [ ]:
print(count_parameters(model))

trainable model parameters: 262541312
all model parameters: 6738546688
percentage of trainable model parameters: 3.90%


# One Shot inferencing

In [ ]:
dataset["train"].features

{'id': Value(dtype='int32', id=None),
 'repo': Value(dtype='string', id=None),
 'path': Value(dtype='string', id=None),
 'func_name': Value(dtype='string', id=None),
 'original_string': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None),
 'code': Value(dtype='string', id=None),
 'code_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'docstring': Value(dtype='string', id=None),
 'docstring_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'sha': Value(dtype='string', id=None),
 'url': Value(dtype='string', id=None)}

In [ ]:
# sample code snippet and it's corresponding problem statement
index=200
code = dataset["train"][index]["code"]
print(code)

function symbolPathSetColor(color, innerColor) {
    if (this.type !== 'image') {
        var symbolStyle = this.style;
        var symbolShape = this.shape;
        if (symbolShape && symbolShape.symbolType === 'line') {
            symbolStyle.stroke = color;
        }
        else if (this.__isEmptyBrush) {
            symbolStyle.stroke = color;
            symbolStyle.fill = innerColor || '#fff';
        }
        else {
            // FIXME 判断图形默认是填充还是描边，使用 onlyStroke ?
            symbolStyle.fill && (symbolStyle.fill = color);
            symbolStyle.stroke && (symbolStyle.stroke = color);
        }
        this.dirty(false);
    }
}


In [ ]:
func_name = dataset["train"][index]["func_name"]
print(func_name)

symbolPathSetColor


In [ ]:
code_tokens = dataset["train"][index]["code_tokens"]
print(code_tokens)

['function', 'symbolPathSetColor', '(', 'color', ',', 'innerColor', ')', '{', 'if', '(', 'this', '.', 'type', '!==', "'image'", ')', '{', 'var', 'symbolStyle', '=', 'this', '.', 'style', ';', 'var', 'symbolShape', '=', 'this', '.', 'shape', ';', 'if', '(', 'symbolShape', '&&', 'symbolShape', '.', 'symbolType', '===', "'line'", ')', '{', 'symbolStyle', '.', 'stroke', '=', 'color', ';', '}', 'else', 'if', '(', 'this', '.', '__isEmptyBrush', ')', '{', 'symbolStyle', '.', 'stroke', '=', 'color', ';', 'symbolStyle', '.', 'fill', '=', 'innerColor', '||', "'#fff'", ';', '}', 'else', '{', '// FIXME 判断图形默认是填充还是描边，使用 onlyStroke ?', 'symbolStyle', '.', 'fill', '&&', '(', 'symbolStyle', '.', 'fill', '=', 'color', ')', ';', 'symbolStyle', '.', 'stroke', '&&', '(', 'symbolStyle', '.', 'stroke', '=', 'color', ')', ';', '}', 'this', '.', 'dirty', '(', 'false', ')', ';', '}', '}']


In [ ]:
original_string = dataset["train"][index]["original_string"]
print(original_string)

function symbolPathSetColor(color, innerColor) {
    if (this.type !== 'image') {
        var symbolStyle = this.style;
        var symbolShape = this.shape;
        if (symbolShape && symbolShape.symbolType === 'line') {
            symbolStyle.stroke = color;
        }
        else if (this.__isEmptyBrush) {
            symbolStyle.stroke = color;
            symbolStyle.fill = innerColor || '#fff';
        }
        else {
            // FIXME 判断图形默认是填充还是描边，使用 onlyStroke ?
            symbolStyle.fill && (symbolStyle.fill = color);
            symbolStyle.stroke && (symbolStyle.stroke = color);
        }
        this.dirty(false);
    }
}


In [ ]:
doc_string = dataset["train"][index]["docstring"]
print(doc_string)

Provide setColor helper method to avoid determine if set the fill or stroke outside


In [ ]:
eval_prompt = """You are a powerful text-to-front-end code model. Your job is to generate front-end code based on the given prompt and context.

You must output the front-end code that fulfills the requirements specified in the prompt.

### Example:
Prompt: Create a responsive navbar with dropdown menus.
Context: Use React and Tailwind CSS for the implementation.
Response:
```jsx
import { useState } from 'react';

export default function Navbar() {
  const [isOpen, setIsOpen] = useState(false);

  return (
    <nav className="bg-gray-800">
      <div className="max-w-7xl mx-auto px-2 sm:px-6 lg:px-8">
        <div className="relative flex items-center justify-between h-16">
          <div className="absolute inset-y-0 left-0 flex items-center sm:hidden">
            <button
              type="button"
              className="inline-flex items-center justify-center p-2 rounded-md text-gray-400 hover:text-white hover:bg-gray-700 focus:outline-none focus:ring-2 focus:ring-inset focus:ring-white"
              aria-controls="mobile-menu"
              aria-expanded="false"
              onClick={() => setIsOpen(!isOpen)}
            >
              <span className="sr-only">Open main menu</span>
              <svg
                className={`${isOpen ? 'hidden' : 'block'} h-6 w-6`}
                xmlns="http://www.w3.org/2000/svg"
                fill="none"
                viewBox="0 0 24 24"
                stroke="currentColor"
                aria-hidden="true"
              >
                <path
                  strokeLinecap="round"
                  strokeLinejoin="round"
                  strokeWidth="2"
                  d="M4 6h16M4 12h16m-7 6h7"
                />
              </svg>
              <svg
                className={`${isOpen ? 'block' : 'hidden'} h-6 w-6`}
                xmlns="http://www.w3.org/2000/svg"
                fill="none"
                viewBox="0 0 24 24"
                stroke="currentColor"
                aria-hidden="true"
              >
                <path
                  strokeLinecap="round"
                  strokeLinejoin="round"
                  strokeWidth="2"
                  d="M6 18L18 6M6 6l12 12"
                />
              </svg>
            </button>
          </div>
          <div className="flex-1 flex items-center justify-center sm:items-stretch sm:justify-start">
            <div className="flex-shrink-0">
              <img
                className="block lg:hidden h-8 w-auto"
                src="https://tailwindui.com/img/logos/workflow-mark-indigo-500.svg"
                alt="Workflow"
              />
              <img
                className="hidden lg:block h-8 w-auto"
                src="https://tailwindui.com/img/logos/workflow-logo-indigo-500-mark-white-text.svg"
                alt="Workflow"
              />
            </div>
            <div className="hidden sm:block sm:ml-6">
              <div className="flex space-x-4">
                <a
                  href="#"
                  className="text-gray-300 hover:bg-gray-700 hover:text-white px-3 py-2 rounded-md text-sm font-medium"
                >
                  Dashboard
                </a>
                <a
                  href="#"
                  className="text-gray-300 hover:bg-gray-700 hover:text-white px-3 py-2 rounded-md text-sm font-medium"
                >
                  Team
                </a>
                <a
                  href="#"
                  className="text-gray-300 hover:bg-gray-700 hover:text-white px-3 py-2 rounded-md text-sm font-medium"
                >
                  Projects
                </a>
                <a
                  href="#"
                  className="text-gray-300 hover:bg-gray-700 hover:text-white px-3 py-2 rounded-md text-sm font-medium"
                >
                  Calendar
                </a>
              </div>
            </div>
          </div>
        </div>
      </div>
    </nav>
  );
}

"""

# Example input for testing the model
example_input = {
    'Prompt': 'Create a checkout page that looks like shopify',
    'Context': 'Use React and Tailwind CSS for the implementation.'
}

final_prompt = f"""
{eval_prompt}

### Input:
{example_input['Prompt']}

### Context:
{example_input['Context']}

### Response:
"""

print(final_prompt)


You are a powerful text-to-front-end code model. Your job is to generate front-end code based on the given prompt and context.

You must output the front-end code that fulfills the requirements specified in the prompt.

### Example:
Prompt: Create a responsive navbar with dropdown menus.
Context: Use React and Tailwind CSS for the implementation.
Response:
```jsx
import { useState } from 'react';

export default function Navbar() {
  const [isOpen, setIsOpen] = useState(false);

  return (
    <nav className="bg-gray-800">
      <div className="max-w-7xl mx-auto px-2 sm:px-6 lg:px-8">
        <div className="relative flex items-center justify-between h-16">
          <div className="absolute inset-y-0 left-0 flex items-center sm:hidden">
            <button
              type="button"
              className="inline-flex items-center justify-center p-2 rounded-md text-gray-400 hover:text-white hover:bg-gray-700 focus:outline-none focus:ring-2 focus:ring-inset focus:ring-white"
       

In [ ]:
inputs = tokenizer(final_prompt, return_tensors="pt").to("cuda")
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=256,
    )[0],
    skip_special_tokens=True
)


dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'BASELINE CODE:\n{code}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


---------------------------------------------------------------------------------------------------
BASELINE CODE:
function symbolPathSetColor(color, innerColor) {
    if (this.type !== 'image') {
        var symbolStyle = this.style;
        var symbolShape = this.shape;
        if (symbolShape && symbolShape.symbolType === 'line') {
            symbolStyle.stroke = color;
        }
        else if (this.__isEmptyBrush) {
            symbolStyle.stroke = color;
            symbolStyle.fill = innerColor || '#fff';
        }
        else {
            // FIXME 判断图形默认是填充还是描边，使用 onlyStroke ?
            symbolStyle.fill && (symbolStyle.fill = color);
            symbolStyle.stroke && (symbolStyle.stroke = color);
        }
        this.dirty(false);
    }
}

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:

You are a powerful text-to-front-end code model. Your job is to generate front-end code based on the giv

# Pre-Processing dataset

In [ ]:
# Adding padding token (in case it's not already present)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# resizing the token embeddings to match the tokenizer
model.resize_token_embeddings(len(tokenizer))

def tokenize_function(example):
  start_prompt = "Generate the code for the following problem statement:\n\n"
  end_prompt = "\n\nResponse: "
  prompt = [start_prompt + problem_statement + end_prompt for problem_statement in example["docstring"]]
  example["input_ids"] = tokenizer(prompt, padding="max_length", max_length=512, truncation=True, return_tensors="pt").input_ids
  example["labels"] = tokenizer(example["code"], padding="max_length", max_length=512, truncation=True, return_tensors="pt").input_ids
  return example

# this dataset is containing 3 different splits (train, validation and test)
tokenized_datasets = dataset.map(tokenize_function, batched=True, batch_size=16) # note - default batch_size=1000

Map:   0%|          | 0/58025 [00:00<?, ? examples/s]

Map:   0%|          | 0/3885 [00:00<?, ? examples/s]

Map:   0%|          | 0/3291 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'input_ids', 'labels'],
        num_rows: 58025
    })
    validation: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'input_ids', 'labels'],
        num_rows: 3885
    })
    test: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'input_ids', 'labels'],
        num_rows: 3291
    })
})

In [ ]:
# removing all not required columns
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'repo', 'path', 'func_name', 'original_string', 'language',
                                                       'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha',
                                                       'url'])

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 58025
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 3885
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 3291
    })
})

In [ ]:
print(tokenized_datasets['train'][0])

{'input_ids': [32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016, 32016

In [ ]:
# sampling data points
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter:   0%|          | 0/58025 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3885 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3291 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 581
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 39
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 33
    })
})

# Seting up the PEFT/LoRA model for Fine-Tuning

note - we can not perform full fine tuning in case of a quantized model. we will have to use some kind of an adapter. for instance we will use `Lora` adapter here.

In [ ]:
from peft import LoraConfig, get_peft_model

# note - global regulaization (no target modules, we can set in case we need!)
lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model.add_adapter(lora_config, adapter_name="adapter_1")

# adding Lora adapter to the original model
peft_model = get_peft_model(model, lora_config)

In [ ]:
# number of trainable parameters
print(count_parameters(peft_model)) # trainable parameters have dropped from 3.9% to 0.25%

trainable model parameters: 16777216
all model parameters: 6772109312
percentage of trainable model parameters: 0.25%


In [ ]:
# fine tuning the model with preprocessed dataset
output_dir = f'./peft-code-generation-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=2e-3,
    num_train_epochs=10,
    logging_steps=10,
    max_steps=500
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"]
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
peft_trainer.train()

peft_model_path="./peft-code-generation-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,6.293800
20,5.082500
30,2.863500
40,3.714600
50,3.231400
60,2.039400
70,4.195400
80,3.660700
90,4.171500
100,3.998600


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


('./peft-code-generation-checkpoint-local/tokenizer_config.json',
 './peft-code-generation-checkpoint-local/special_tokens_map.json',
 './peft-code-generation-checkpoint-local/tokenizer.model',
 './peft-code-generation-checkpoint-local/added_tokens.json',
 './peft-code-generation-checkpoint-local/tokenizer.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
checkpoint_path = "/content/drive/MyDrive/CodeLLama_FineTuning/peft-code-generation-checkpoint-final"
peft_trainer.model.save_pretrained(checkpoint_path)
tokenizer.save_pretrained(checkpoint_path)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


('/content/drive/MyDrive/CodeLLama_FineTuning/peft-code-generation-checkpoint-final/tokenizer_config.json',
 '/content/drive/MyDrive/CodeLLama_FineTuning/peft-code-generation-checkpoint-final/special_tokens_map.json',
 '/content/drive/MyDrive/CodeLLama_FineTuning/peft-code-generation-checkpoint-final/tokenizer.model',
 '/content/drive/MyDrive/CodeLLama_FineTuning/peft-code-generation-checkpoint-final/added_tokens.json',
 '/content/drive/MyDrive/CodeLLama_FineTuning/peft-code-generation-checkpoint-final/tokenizer.json')